The purpose of this notebook is to answer the questions put forth here:

https://docs.google.com/document/d/1Tu8-XHeOP9LHn6BmkOEHk__p1JaFNdb6jCw2UQc-vA8/edit

- Mark Halverson
- Last updated: March 16, 2021

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from survey_utils.change_column_names import change_column_names

In [ ]:
df = pd.read_csv("data/BCMT Membership Survey Anonymized Responses.csv")

In [ ]:
df = change_column_names(df)

In [ ]:
# print out the column names
#df.columns

# Apply some string operations to fill gaps and ensure consistency in letter cases

In [ ]:
# trim white spaces on all text in df
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
# lower case some of the responses
df["activities"] = df["activities"].str.lower()
df["primary activity"] = df["primary activity"].str.lower()
df["membership reason"] = df["membership reason"].str.lower()
df["bcmt.org use reason"] = df["bcmt.org use reason"].str.lower()
df["mobile apps used"] = df["mobile apps used"].str.lower()

In [ ]:
# gap fill
print(f'Website use reason has {df["bcmt.org use reason"].isna().sum()} null values')
print(f'Mobile apps used has {df["mobile apps used"].isna().sum()} null values')
print(f'Membership reason has {df["membership reason"].isna().sum()} null values')
print(f'Location has {df["location"].isna().sum()} null values')

df["bcmt.org use reason"].fillna("no answer provided", inplace=True)
df["mobile apps used"].fillna("no answer provided", inplace=True)
df["membership reason"].fillna("no answer provided", inplace=True)
df["location"].fillna("no answer provided", inplace=True)

In [ ]:
# fix up some compound words and spelling mistakes
df["membership reason"] = df["membership reason"].str.replace("camp site", "campsite", regex=False)
df["bcmt.org use reason"] = df["bcmt.org use reason"].str.replace("kajak", "kayak", regex=False)
df["mobile apps used"] = df["mobile apps used"].str.replace("garmon", "garmin", regex=False)

In [ ]:
# fix up Washington state place names
matches = ["Washington","WA","Olympic"]
matches = "|".join(matches)
mask = df["location"].str.contains(matches, regex=True)
#df.loc[mask,"location"]
df.loc[mask,"location"] = "Washington State"

In [ ]:
# fix up Vancouver place names
matches = ["We lived in Vancouver|West Vancouver"]
matches = "|".join(matches)
mask = df["location"].str.contains(matches, regex=True)
#df.loc[mask,"location"]
df.loc[mask,"location"] = "Vancouver Coast & Mountains"

In [ ]:
# fix up Vancouver Island / Gulf Islands place names
matches = ["Salt Spring"]
matches = "|".join(matches)
mask = df["location"].str.contains(matches, regex=True)
#df.loc[mask,"location"]
df.loc[mask,"location"] = "Vancouver Island"

In [ ]:
# fix up Vancouver Island / Gulf Islands place names
matches = ["Calgary"]
matches = "|".join(matches)
mask = df["location"].str.contains(matches, regex=True)
#df.loc[mask,"location"]
df.loc[mask,"location"] = "Alberta"

In [ ]:
# fix up Oregon place names
matches = ["Oregon"]
matches = "|".join(matches)
mask = df["location"].str.contains(matches, regex=True)
#df.loc[mask,"location"]
df.loc[mask,"location"] = "Oregon"

-------------------------------

# What are the characteristics of the respondents to cite “support”, “contribute“, “help” as a membership reason?  

In [ ]:
#df["membership reason"].head(10).to_list()

In [ ]:
matches = ["support","contribute","help"]
matches = "|".join(matches)
matches

In [ ]:
mask = df["membership reason"].str.contains(matches, regex=True)
mask.sum()

In [ ]:
df_support = df.loc[mask]

## supports broken down by single category

In [ ]:
df.columns

In [ ]:
cols = ['age group', 'membership duration','bcmt.org use reason',
        'submitted scr','gender','location']

In [ ]:
df_dict = {}
for col in cols:
    df1 = df_support[col].value_counts()
    df2 = df[col].value_counts()
    the_df = pd.concat([df1, df2], axis=1)
    the_df.columns = ["supporters","all members"]
    the_df["percent"] = 100*df1/df2
    the_df.fillna(0, inplace=True)
    the_df["supporters"] = the_df["supporters"].astype(int)
    df_dict[col] = the_df
    print(col)
    print(the_df, end=2*"\n")

In [ ]:
# trim down the bcmt.org use reason category
df_dict["bcmt.org use reason"] = df_dict["bcmt.org use reason"].sort_values(by="supporters", ascending=False).head(8)

### now the plotting

In [ ]:
fig, axs = plt.subplots(6,1, figsize=(12,40))
#fig.suptitle('Vertically stacked subplots')

for k, key in enumerate(df_dict):
    
    df_dict[key].sort_values(by="percent", inplace=True)
    
    axs[k].barh(df_dict[key].index, df_dict[key]["percent"])
    
    axs[k].set_title(key, fontsize=14)
    axs[k].set_xlabel("Percentage of respondents citing support", fontsize=14)
    axs[k].tick_params(axis='both', which='major', labelsize=14)
    
    # set up some labels for the bars:  
    x = df_dict[key]["percent"].values
    y = np.arange(0,len(x))
    txt = df_dict[key]["supporters"].astype(str) + "/" + df_dict[key]["all members"].astype(str)
    
    axs[k].set_xlim(0,1.1*max(x))
    
    # now add label showing n_supporters and n_respondents
    for j in range(0,len(x)): 
        axs[k].text(x[j] + 2, y[j], txt[j], color='black', fontweight='normal', va='center', fontsize=14)

plt.tight_layout()

plt.savefig("/Users/Mark/Desktop/who_supports_the_bcmt.png",facecolor='w')
!open ~/Desktop/who_supports_the_bcmt.png

## supporters broken down by age group and membership duration

In [ ]:
n_support = df.loc[mask].groupby(by=["age group","membership duration"]).count()["timestamp"].sort_values(ascending=False)
n_support.name = "number citing support"

In [ ]:
n_total = df.groupby(by=["age group","membership duration"]).count()["timestamp"].sort_values(ascending=False)
n_total.name = "total number"

In [ ]:
support_df = pd.concat([n_support, n_total], axis=1)

In [ ]:
support_df["as_percent"] = 100*n_support/n_total

In [ ]:
support_df["as_percent"] = support_df["as_percent"].round(1)

In [ ]:
support_df.fillna(0, inplace=True)

In [ ]:
support_df["number citing support"] = support_df["number citing support"].astype(int)

In [ ]:
support_df

In [ ]:
support_df["as_percent"].sort_values().plot.barh(figsize=(10,10));

In [ ]:
new_df = support_df.groupby("age group").sum()[["number citing support","total number"]]
new_df["percentage"] = np.round(100*new_df["number citing support"]/new_df["total number"],1)
new_df

In [ ]:
support_df.groupby("membership duration").sum()[["number citing support","total number"]]

In [ ]:
df.loc[mask,"age group"].value_counts()/df["age group"].value_counts()